# Gyro relations

## Angus+2019
* https://ui.adsabs.harvard.edu/abs/2019AJ....158..173A/abstract

In [1]:
# %load init_ipynb
import os
import sys
import matplotlib.pyplot as pl

sys.path.insert(0, '../code')
assert os.environ["CONDA_DEFAULT_ENV"] == 'wakai'

plotpar = {'axes.labelsize': 16,
           'font.size': 16,
           'legend.fontsize': 16,
           'xtick.labelsize': 16,
           'ytick.labelsize': 16,
#            'text.usetex': False,
           'xtick.direction': 'in',
           'ytick.direction': 'in'
           }
pl.rcParams.update(plotpar)

# known young host stars

In [2]:
from utils import get_nexsci_data

df_nexsci = get_nexsci_data(table_name="ps", clobber=False)

Column definitions:  https://exoplanetarchive.ipac.caltech.edu/docs/API_PS_columns.html


/home/jp/github/research/project/wakai/notebooks/../code/utils.py:39: DtypeWarning: Columns (3,4,50,65,148,153,183,206,211,237,242) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nexsci = pd.read_csv(fp)


Loaded:  ../data/nexsci_ps.csv


In [3]:
from utils import get_relative_err_index

idx = (df_nexsci.st_age<1) & (df_nexsci.default_flag==1) #& (df_nexsci.disc_facility=='Kepler') 
idx2 = get_relative_err_index(df_nexsci[idx], par='st_age', rel_err=0.2)

df_young = df_nexsci[idx][idx2]
young_hoststar_names = df_young.hostname.unique()
len(young_hoststar_names), young_hoststar_names

(21,
 array(['WASP-189', 'Qatar-4', 'Qatar-5', 'Qatar-3', 'HIP 94235',
        'DS Tuc A', 'HD 110082', 'HD 63433', 'K2-100', 'Kepler-1643',
        'Kepler-1928', 'Kepler-411', 'K2-102', 'TOI-2048', 'TOI-1860',
        'K2-25', 'TOI-1227', 'Kepler-970', 'K2-33', 'V1298 Tau',
        'HIP 67522'], dtype=object))